In [1]:
import pandas as pd
from matplotlib import pylab
from scipy.stats import stats, probplot
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib as plt
import scipy
import numpy as np
from statsmodels.stats.weightstats import *
%pylab inline

Populating the interactive namespace from numpy and matplotlib


c:\my files\python\pythonproject\lib\site-packages\IPython\core\magics\pylab.py:159: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [2]:
data1=[1]*10+[0]*24
data2=[1]*4+[0]*12

In [20]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)

    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2
    P = float(p1*n1 + p2*n2) / (n1 + n2)

    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")

    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))

    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [21]:
p=proportions_diff_z_test(proportions_diff_z_stat_ind(data1, data2),'greater')
round(p,4)

0.3729

In [23]:
data=pd.read_csv('banknotes.txt',sep='\t')
data.head()

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


In [24]:
X=data[data.columns[:-1]]
y=data.real

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state=1)
X_test.shape

(50, 6)

In [26]:
from sklearn.linear_model import LogisticRegression

In [29]:
clf1 = LogisticRegression(random_state=1).fit(X_train[['X1', 'X2', 'X3']], y_train)
clf1_pred = clf1.predict(X_test[['X1', 'X2', 'X3']])

In [30]:
clf2 = LogisticRegression(random_state=1).fit(X_train[['X4', 'X5', 'X6']], y_train)
clf2_pred = clf2.predict(X_test[['X4', 'X5', 'X6']])

In [31]:
t=np.sum(np.array([clf1_pred!=y_test]))
f=len(y_test)-t
ar1=[1]*t+[0]*f

In [32]:
t=np.sum(np.array([clf2_pred!=y_test]))
f=len(y_test)-t
ar2=[1]*t+[0]*f

In [34]:
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = list(zip(sample1, sample2))
    n = len(sample)

    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])

    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = list(zip(sample1, sample2))
    n = len(sample)

    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])

    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [35]:
proportions_diff_z_stat_rel(ar1, ar2)

3.312945782245396

In [39]:
proportions_diff_z_test(proportions_diff_z_stat_rel(ar1, ar2))

0.0009231887438696518

In [40]:
proportions_diff_confint_rel(ar1, ar2)

(0.0735106275786684, 0.2864893724213316)

In [43]:
1 - scipy.stats.norm.cdf(1.65)

0.0494714680336481